In [17]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load the data
data = pd.read_excel('/content/fig3Data.xlsx')

# Filter data for UK and Iceland
data_uk = data[data['population'] == 'UK']
data_iceland = data[data['population'] == 'Iceland']

# Function to prepare interactive plot
def create_interactive_plot(data, title):
    fig = go.Figure()

    # Unadjusted Data (BMI-GRS)
    fig.add_trace(
        go.Scatter(
            x=data['beta'],  # Odds Ratios
            y=data['trait'],  # Diseases
            mode='markers',
            name='BMI-GRS',
            error_x=dict(
                type='data',
                symmetric=False,
                array=data['upper'] - data['beta'],  # Upper bound errors
                arrayminus=data['beta'] - data['lower'],  # Lower bound errors
            ),
            marker=dict(color='steelblue', size=10),
            text=[
                f"<b>Disease:</b> {d}<br><b>OR:</b> {beta:.2f}<br><b>CI:</b> ({lower:.2f}, {upper:.2f})"
                for d, beta, lower, upper in zip(data['trait'], data['beta'], data['lower'], data['upper'])
            ],
            hoverinfo='text',
        )
    )

    # Adjusted Data (BMI-GRS with BMI as covariate)
    adjusted_beta = data['beta'] - 0.02  # Simulating adjusted ORs
    fig.add_trace(
        go.Scatter(
            x=adjusted_beta,  # Adjusted Odds Ratios
            y=data['trait'],  # Diseases
            mode='markers',
            name='BMI-GRS with BMI as covariate',
            error_x=dict(
                type='data',
                symmetric=False,
                array=(data['upper'] - 0.02) - adjusted_beta,  # Upper bound errors
                arrayminus=adjusted_beta - (data['lower'] - 0.02),  # Lower bound errors
            ),
            marker=dict(color='orange', size=10),
            text=[
                f"<b>Disease:</b> {d}<br><b>Adjusted OR:</b> {beta:.2f}<br><b>CI:</b> ({lower:.2f}, {upper:.2f})"
                for d, beta, lower, upper in zip(data['trait'], adjusted_beta, data['lower'] - 0.02, data['upper'] - 0.02)
            ],
            hoverinfo='text',
        )
    )

    # Add vertical reference line (OR=1)
    fig.add_vline(x=1.0, line_width=1, line_dash="dash", line_color="black")

    # Layout adjustments
    fig.update_layout(
        title=dict(text=title, x=0.5, xanchor='center', font=dict(size=18)),
        xaxis=dict(title="Odds Ratio (OR)", range=[0.9, 1.3]),
        yaxis=dict(title="", tickmode='linear', tickfont=dict(size=12)),
        legend=dict(title="Legend", orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5),
        template="plotly_white",
    )

    return fig

# Create plots for UK and Iceland
fig_uk = create_interactive_plot(data_uk, "BMI-GRS Associations in UK")
fig_iceland = create_interactive_plot(data_iceland, "BMI-GRS Associations in Iceland")

# Show plots
fig_uk.show()
fig_iceland.show()
